In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split


In [35]:
data = pd.read_csv(r'C:\Users\User\newdoc_data_add.CSV', encoding='cp949')

In [3]:
data 

,conversation,label
0,요즘 계속해서 피로하고 기운이 없어서 걱정이에요.,피로감
1,최근에 식욕이 많이 없어져서 걱정이에요.,식욕감퇴
2,소화가 잘 안되는 것 같기도 하고,소화불량
3,가끔 배도 아파요,복통
4,가끔 복부 통증도 있어요.,복통
...,...,...
1651,몸살 같이 아파요,몸살
1652,몸살 같이 몸이 지치고 힘들어요,몸살
1653,몸살 기운 때문에 힘들어요,몸살
1654,몸살 났어요,몸살


In [15]:
#데이터 전처리

# 대화와 레이블 추출
conversations = data['conversation'].tolist()
labels = data['label'].tolist()

# 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(conversations)
sequences = tokenizer.texts_to_sequences(conversations)

# 시퀀스 패딩
max_len = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# 레이블 이진 벡터 변환
label_binarizer = LabelBinarizer()
encoded_labels = label_binarizer.fit_transform(labels)
num_classes = len(label_binarizer.classes_)

# 데이터 분할
train_sequences, test_sequences, train_labels, test_labels = train_test_split(
    padded_sequences, encoded_labels, test_size=0.2, random_state=42 ,stratify=labels)

In [29]:
# 모델 

import joblib
from keras.models import load_model

from keras.callbacks import EarlyStopping 
# 모델 구성
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_len))
model.add(LSTM(units=64))
model.add(Dense(units=num_classes, activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# 모델 학습
model.fit(train_sequences, train_labels, epochs=1000, batch_size=16, validation_data=(test_sequences, test_labels), callbacks=[early_stopping])


# 학습된 모델 저장
model.save('doc_model.h5')

Epoch 1/1000
83/83 [==============================] - 6s 33ms/step - loss: 3.2044 - accuracy: 0.0763 - val_loss: 3.1258 - val_accuracy: 0.1175
Epoch 2/1000
83/83 [==============================] - 1s 12ms/step - loss: 2.8302 - accuracy: 0.2613 - val_loss: 2.3403 - val_accuracy: 0.3946
Epoch 3/1000
83/83 [==============================] - 1s 15ms/step - loss: 1.6341 - accuracy: 0.6405 - val_loss: 1.3441 - val_accuracy: 0.7199
Epoch 4/1000
83/83 [==============================] - 1s 12ms/step - loss: 0.7894 - accuracy: 0.8784 - val_loss: 0.9127 - val_accuracy: 0.7982
Epoch 5/1000
83/83 [==============================] - 1s 11ms/step - loss: 0.4128 - accuracy: 0.9517 - val_loss: 0.6638 - val_accuracy: 0.8253
Epoch 6/1000
83/83 [==============================] - 1s 12ms/step - loss: 0.2303 - accuracy: 0.9796 - val_loss: 0.5709 - val_accuracy: 0.8645
Epoch 7/1000
83/83 [==============================] - 1s 12ms/step - loss: 0.1435 - accuracy: 0.9909 - val_loss: 0.5037 - val_accuracy: 0.8795

In [30]:
test_loss, test_accuracy = model.evaluate(test_sequences, test_labels)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

11/11 [==============================] - 0s 7ms/step - loss: 0.4196 - accuracy: 0.8946
Test Loss: 0.4196326434612274
Test Accuracy: 0.8945783376693726


In [36]:
import joblib

model = joblib.load(r'C:\Users\User\BE\34\web\model\doc_model.pkl')
max_len=14
tokenizer=joblib.load(r"C:\Users\User\BE\34\web\model\tokenizer.joblib")
label_binarizer = joblib.load(r"C:\Users\User\BE\34\web\model\labelbinarizer.joblib")


In [6]:
!pip install konlpy pandas seaborn gensim wordcloud python-mecab-ko wget

In [59]:
from mecab import MeCab
from konlpy.tag import Kkma
# def tokenize_sentences(text):
#     mecab = MeCab()
#     tokens = mecab.nouns(text)
#     return tokens
def tokenize_sentences(text):
    kkma = Kkma()
    sentences = kkma.sentences(text)
    return sentences

# new_texts = ['갑자기 심장이 빨리 뛰어요.',
#              '공복 혈압도 많이 낮은 것 같아요.',
#              '빈혈 증상도 같이 오는 기분이에요  . 그리고 배가 너무 아파요',
#              '소화가 너무 안 되는 느낌이 들어요.',
#              '입맛이 없어서 아무것도 안 먹고 있어요.',
#              '혹시 머리도 너무 아픈데 두통약도 처방해 주실 수 있나요.',
#              '속이 울렁거리고 식욕이 없어요. 그래서 살이 3kg가 빠졌어요.',
#              '가끔 어지러운데 심하진 않아요.',
#              '아니요. 똑같아요.',
#              '네 감사합니다.']

new_texts = ['요즘에 배가 자주 아파서 식욕이 많이 줄었어요',
             '식사 후에 위가 불편해요. 또 몸무게도 많이 줄었어요','네 감사합니다'
             ]


tokenized_sentences = []
for text in new_texts:
    sentences = tokenize_sentences(text)
    tokenized_sentences.extend(sentences)

new_sequences = tokenizer.texts_to_sequences(tokenized_sentences)
new_padded_sequences = pad_sequences(new_sequences, maxlen=14)
predictions = model.predict(new_padded_sequences)

decoded_labels = label_binarizer.inverse_transform(predictions)
print(decoded_labels)


1/1 [==============================] - 0s 35ms/step
['식욕감퇴' '소화불량' '체중감소' '두통']


In [30]:
tokenized_sentences

['심장',
 '공복',
 '혈압',
 '것',
 '빈혈',
 '증상',
 '기분',
 '배',
 '소화',
 '느낌',
 '입맛',
 '아무것',
 '머리',
 '두통약',
 '처방',
 '수',
 '속',
 '식욕',
 '살',
 '데',
 '감사']

In [33]:
tokenized_sentences

['갑자기 심장이 빨리 뛰어요.',
 '공복 혈압도 많이 낮은 것 같아요.',
 '빈혈 증상도 같이 오는 기분이에요 .',
 '그리고 배가 너무 아파요',
 '소화가 너무 안 되는 느낌이 들어요.',
 '입맛이 없어서 아무것도 안 먹고 있어요.',
 '혹시 머리도 너무 아픈데 두통약도 처방해 주실 수 있나요.',
 '속이 울렁거리고 식욕이 없어요.',
 '그래서 살이 3kg 가 빠졌어요.',
 '가끔 어지러운데 심하진 않아요.',
 '아니요. 똑같아요.',
 '네 감사합니다.']

In [66]:

label_names = label_binarizer.classes_
top_k = 10 

top_k_indices = np.argsort(-predictions, axis=1)[:, :top_k]
top_k_probabilities = np.take_along_axis(predictions, top_k_indices, axis=1)

symptom = []  
for i in range(predictions.shape[0]):
    print(f"text {i+1}:")
    for j in range(top_k):
        label_index = top_k_indices[i, j]
        label_name = label_names[label_index]
        prob = top_k_probabilities[i, j]
        print(f"Label {j+1}: {label_name}, Probability: {prob}")
        
        if prob > 0.3:
            symptom.append(label_name)
    
    # print()

print("Symptoms:")
print(symptom)

text 1:
Label 1: 식욕감퇴, Probability: 0.560653567314148
Label 2: 복통, Probability: 0.3758377134799957
Label 3: 속쓰림, Probability: 0.018018782138824463
Label 4: 식욕증가, Probability: 0.017916684970259666
Label 5: 요통, Probability: 0.00590095529332757
Label 6: 체중감소, Probability: 0.0039043889846652746
Label 7: 발진, Probability: 0.0026953592896461487
Label 8: 체중증가, Probability: 0.0023000496439635754
Label 9: 발열, Probability: 0.002281367313116789
Label 10: 변비, Probability: 0.0021990041714161634
text 2:
Label 1: 소화불량, Probability: 0.9668372869491577
Label 2: 속쓰림, Probability: 0.013055309653282166
Label 3: 요통, Probability: 0.005066601559519768
Label 4: 식욕감퇴, Probability: 0.004898895975202322
Label 5: 발진, Probability: 0.0030602861661463976
Label 6: 변비, Probability: 0.00206229486502707
Label 7: 호흡곤란, Probability: 0.0016051407437771559
Label 8: 식욕증가, Probability: 0.0008026182185858488
Label 9: 신경통, Probability: 0.0007015858427621424
Label 10: 몸살, Probability: 0.0006313872872851789
text 3:
Label 1: 체중감소, 

In [ ]:
# 모델 저장
joblib.dump(label_binarizer, r'C:\Users\User\MLOps\model\label_binarizer2.pkl')
joblib.dump(tokenizer, r'C:\Users\User\MLOps\model\tokenizer2.pkl')
joblib.dump(model, 'doc_model.pkl')